In [1]:
# 批量转换json标注文件to mask
import os
path = 'C:\\Users\\hkc\\Desktop\\meter_img\\json' # path为json文件存放的路径 
json_file = os.listdir(path) 
os.system("activate labelme") 
for file in json_file: 
    os.system("labelme_json_to_dataset.exe %s"%(path + '/' + file))

In [ ]:
# 批量改文件名
import os
# path=input('请输入文件路径(结尾加上/)：')
# path = 'imgs/'

#获取该目录下所有文件，存入列表中
fileList=os.listdir(path)

n=0
for i in fileList:
    
    #设置旧文件名（就是路径+文件名）
    oldname=path+ os.sep + fileList[n]   # os.sep添加系统分隔符
    
    #设置新文件名
    newname=path + os.sep +'ture'+str(n+1)+'.jpg'
    
    os.rename(oldname,newname)   #用os模块中的rename方法对文件改名
    print(oldname,'======>',newname)
    
    n+=1

In [10]:
# 批量修改img和mask文件名和提取出来
import os
import glob

if __name__ == "__main__":
    path = 'C:\\Users\\hkc\\Desktop\\meter_img\\json'
    imgs_paths = glob.glob('C:\\Users\\hkc\\Desktop\\meter_img\\json\\*_json')
    print(imgs_paths)
    save_img_path = 'C:\Users\hkc\Desktop\meter_img\train_img'
    save_mask_path = 'C:\Users\hkc\Desktop\meter_img\train_mask'
    for image_path in imgs_paths:

        print(image_path)


['C:\\Users\\hkc\\Desktop\\meter_img\\json\\meter_img01_json', 'C:\\Users\\hkc\\Desktop\\meter_img\\json\\meter_img03_json', 'C:\\Users\\hkc\\Desktop\\meter_img\\json\\meter_img05_json', 'C:\\Users\\hkc\\Desktop\\meter_img\\json\\meter_img06_json', 'C:\\Users\\hkc\\Desktop\\meter_img\\json\\meter_img07_json', 'C:\\Users\\hkc\\Desktop\\meter_img\\json\\meter_img08_json', 'C:\\Users\\hkc\\Desktop\\meter_img\\json\\meter_img10_json', 'C:\\Users\\hkc\\Desktop\\meter_img\\json\\meter_img11_json', 'C:\\Users\\hkc\\Desktop\\meter_img\\json\\meter_img12_json', 'C:\\Users\\hkc\\Desktop\\meter_img\\json\\meter_img13_json', 'C:\\Users\\hkc\\Desktop\\meter_img\\json\\meter_img14_json', 'C:\\Users\\hkc\\Desktop\\meter_img\\json\\meter_img15_json', 'C:\\Users\\hkc\\Desktop\\meter_img\\json\\meter_img16_json', 'C:\\Users\\hkc\\Desktop\\meter_img\\json\\meter_img17_json', 'C:\\Users\\hkc\\Desktop\\meter_img\\json\\meter_img18_json', 'C:\\Users\\hkc\\Desktop\\meter_img\\json\\meter_img19_json', 'C:\\Us

In [1]:
# 批量修改mask
import cv2
import os
import numpy as np
import glob
import random
# from PIL import Image

if __name__ == "__main__":
    imgs_paths = glob.glob('mask/*.png')
    for image_path in imgs_paths:
        savepath = image_path.replace('mask', 'train_mask')
        label = cv2.imread(image_path)
        label = cv2.cvtColor(label, cv2.COLOR_BGR2GRAY)
        print(label.shape)
        # print(savepath)
        indax_point = np.where(label == 38)
        label[indax_point] = 1
        indax_point = np.where(label == 75)
        label[indax_point] = 2
        cv2.imwrite(savepath, label)
    print('end')


(509, 502)
(482, 497)
(287, 279)
(529, 552)
(418, 372)
(477, 476)
(330, 331)
(281, 283)
(327, 323)
(384, 398)
(288, 284)
(328, 341)
(756, 777)
(279, 287)
(414, 430)
(347, 345)
(431, 421)
(504, 510)
(471, 466)
(737, 743)
(329, 342)
(541, 545)
(387, 390)
(409, 416)
(336, 330)
(287, 280)
(342, 343)
(323, 324)
(421, 419)
(336, 334)
(327, 338)
(349, 353)
(441, 446)
(343, 342)
(343, 342)
end


In [19]:
#测试修改
import cv2
import os
import numpy as np
import glob
import random

if __name__ == "__main__":
    imgs_paths = glob.glob('train_mask/*.png')
    for image_path in imgs_paths:
        label = cv2.imread(image_path,0)
        print(label.shape)
    print('end')


(249, 251)
(313, 314)
(209, 218)
(817, 833)
(844, 839)
(768, 768)
(508, 512)
(282, 281)
(483, 479)
(206, 207)
(112, 113)
(274, 275)
(476, 479)
(458, 461)
(247, 240)
(361, 366)
end


In [ ]:
# 批量修复train.txt, val.txt中图像路径 imgpath_repair.py
import os

def changepath(path,topath):
    file = open(path, 'r')
    lines = []
    for i in file:
        lines.append(i)  #逐行将文本存入列表lines中
    file.close()

    new = []
    for line in lines:  # 逐行遍历
        line = line.replace('images/train/','')
        line = line.replace('images/val/','')
        p = 0  # 定义计数指针
        for bit in line:  # 对每行进行逐个字遍历
            if bit == '.':   # 遇到0时进行处理（我们可以修改成我们想要的定位）
                line = line[:p]
                break
            else:
                p = p + 1  # 如果bit不是空格，指针加一
        new.append(line)
    
    file_write = open(topath, 'w')
    for var in new:
        file_write.writelines(var)
        file_write.writelines('\n')
    file_write.close()


if __name__ == '__main__':
    changepath('E:/sc/image_data/meter/meter_seg_voc/ImageSet/Segmentation/val_yuan.txt'
               , 'E:/sc/image_data/meter/meter_seg_voc/ImageSet/Segmentation/val.txt')
